In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.stats import zscore
import math

In [4]:
MSA_ZScore = pd.read_csv('/content/drive/MyDrive/03 Courses/02 Spring/2022 CUSP - Capstone project/05 Pipeline/01 MSA/01 Data/03_MSA_ZScore_ManualCheck.csv')
MSA_ZScore = MSA_ZScore[['GEO_ID','Total_housing_units','kitch_zscore','plumbing_zscore','age_zscore','income_zscore']]
MSA_ZScore

,GEO_ID,Total_housing_units,kitch_zscore,plumbing_zscore,age_zscore,income_zscore
0,1400000US48085030100,2242,0.513433,0.462763,0.576587,-0.265092
1,1400000US48085030201,1136,0.513433,0.462763,0.832801,0.183436
2,1400000US48085030202,1254,0.513433,0.462763,0.768747,0.840757
3,1400000US48085030203,6745,0.513433,0.462763,1.345228,0.640594
4,1400000US48085030301,2499,-0.201051,-0.447178,1.409282,0.872110
...,...,...,...,...,...,...
1306,1400000US48497150403,1519,0.086000,-0.573621,0.512533,-0.156722
1307,1400000US48497150500,2254,-0.494751,0.462763,-0.640429,-0.793669
1308,1400000US48497150601,3465,0.326053,-0.193497,0.320373,-0.408530
1309,1400000US48497150602,2681,0.362073,0.006056,0.896854,-0.168111


In [6]:
z_score_mult = MSA_ZScore.loc[MSA_ZScore.index.repeat(MSA_ZScore['Total_housing_units'])]
z_score_mult.reset_index(drop = True, inplace = True)
z_score_mult.drop(columns=['Total_housing_units','GEO_ID'], inplace = True)
z_score_mult

,kitch_zscore,plumbing_zscore,age_zscore,income_zscore
0,0.513433,0.462763,0.576587,-0.265092
1,0.513433,0.462763,0.576587,-0.265092
2,0.513433,0.462763,0.576587,-0.265092
3,0.513433,0.462763,0.576587,-0.265092
4,0.513433,0.462763,0.576587,-0.265092
...,...,...,...,...
2764973,-0.717628,-0.421657,0.704694,-0.508215
2764974,-0.717628,-0.421657,0.704694,-0.508215
2764975,-0.717628,-0.421657,0.704694,-0.508215
2764976,-0.717628,-0.421657,0.704694,-0.508215


In [7]:
z_score_arr = z_score_mult.to_numpy()
z_score_arr

array([[ 0.51343251,  0.46276292,  0.57658685, -0.26509239],
       [ 0.51343251,  0.46276292,  0.57658685, -0.26509239],
       [ 0.51343251,  0.46276292,  0.57658685, -0.26509239],
       ...,
       [-0.7176278 , -0.42165683,  0.70469375, -0.50821475],
       [-0.7176278 , -0.42165683,  0.70469375, -0.50821475],
       [-0.7176278 , -0.42165683,  0.70469375, -0.50821475]])

In [9]:
# PCA
pca_msa = PCA()
pca_msa.fit(z_score_arr)
print(pca_msa.components_)

[[-0.34277792 -0.38493289 -0.58819572 -0.62318197]
 [-0.62405696 -0.58455079  0.41019681  0.3171622 ]
 [ 0.69258854 -0.68155915  0.19045803 -0.13972824]
 [ 0.11566038 -0.21355132 -0.67044024  0.70109086]]


In [15]:
# EIGENVECTORS

kitch      =  abs(pca_msa.components_[0][0])
plumbing   =  abs(pca_msa.components_[0][1])
age        =  abs(pca_msa.components_[0][2])
income     =  abs(pca_msa.components_[0][3])

In [16]:
k = 1/(math.sqrt(age**2 + income**2 + kitch**2 + plumbing**2))
k

1.0

In [17]:
MSA_ZScore_PCA = MSA_ZScore.copy()
MSA_ZScore_PCA['PCA_weighted_score'] = (k*age*MSA_ZScore_PCA['age_zscore']) + (k*kitch*MSA_ZScore_PCA['kitch_zscore']) + (k*plumbing*MSA_ZScore_PCA['plumbing_zscore']) + (k*income*MSA_ZScore_PCA['income_zscore'])
MSA_ZScore_PCA

,GEO_ID,Total_housing_units,kitch_zscore,plumbing_zscore,age_zscore,income_zscore,PCA_weighted_score
0,1400000US48085030100,2242,0.513433,0.462763,0.576587,-0.265092,0.528071
1,1400000US48085030201,1136,0.513433,0.462763,0.832801,0.183436,0.958290
2,1400000US48085030202,1254,0.513433,0.462763,0.768747,0.840757,1.330244
3,1400000US48085030203,6745,0.513433,0.462763,1.345228,0.640594,1.544590
4,1400000US48085030301,2499,-0.201051,-0.447178,1.409282,0.872110,1.131367
...,...,...,...,...,...,...,...
1306,1400000US48497150403,1519,0.086000,-0.573621,0.512533,-0.156722,0.012477
1307,1400000US48497150500,2254,-0.494751,0.462763,-0.640429,-0.793669,-0.862755
1308,1400000US48497150601,3465,0.326053,-0.193497,0.320373,-0.408530,-0.028866
1309,1400000US48497150602,2681,0.362073,0.006056,0.896854,-0.168111,0.549204


In [18]:
# check if the weighted Z Score has zero sum
(MSA_ZScore_PCA['PCA_weighted_score'] * MSA_ZScore_PCA['Total_housing_units']).sum()

1.076841726899147e-09

In [19]:
from google.colab import files 
MSA_ZScore_PCA.to_csv('04_MSA_ZScore_PCA.csv', index=False) 
files.download('04_MSA_ZScore_PCA.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>